In [192]:
import pandas as pd
import numpy as np

#pd.set_option('display.max_rows', None)
#coaching_time_compare

In [193]:
#fitbit data
wear_time = pd.read_csv("D:\\EIC-Code\\00-Python\\2024-08-Objective-identification-on-mental-health_CNN_LSTM\\data\\fitbitWearTimeViaHR_merged.csv")
dailyactivity = pd.read_csv("D:\\EIC-Code\\00-Python\\2024-08-Objective-identification-on-mental-health_CNN_LSTM\\data\\dailyActivity_merged.csv")
dailyFitbitActiveZoneMinutes = pd.read_csv("D:\\EIC-Code\\00-Python\\2024-08-Objective-identification-on-mental-health_CNN_LSTM\\data\\dailyFitbitActiveZoneMinutes_merged.csv")
dailyHRV = pd.read_csv("D:\\EIC-Code\\00-Python\\2024-08-Objective-identification-on-mental-health_CNN_LSTM\\data\\fitbitDailyHRV_merged.csv")
fitbitBreathingRate = pd.read_csv("D:\\EIC-Code\\00-Python\\2024-08-Objective-identification-on-mental-health_CNN_LSTM\\data\\fitbitBreathingRate_merged.csv")
fitbitSkinTemperature = pd.read_csv("D:\\EIC-Code\\00-Python\\2024-08-Objective-identification-on-mental-health_CNN_LSTM\\data\\fitbitSkinTemperature_merged.csv")
sleepDay = pd.read_csv("D:\\EIC-Code\\00-Python\\2024-08-Objective-identification-on-mental-health_CNN_LSTM\\data\\sleepDay_merged.csv")
heartrate_15min = pd.read_csv("D:\\EIC-Code\\00-Python\\2024-08-Objective-identification-on-mental-health_CNN_LSTM\\data\\heartrate_15min_merged.csv")
sleepStageLogInfo = pd.read_csv("D:\\EIC-Code\\00-Python\\2024-08-Objective-identification-on-mental-health_CNN_LSTM\\data\\sleepStagesDay_merged.csv")

#survey data
survey_data = pd.read_csv("D:\\EIC-Code\\00-Python\\2024-08-Objective-identification-on-mental-health_CNN_LSTM\\data\\ABriefInterventionTo_DATA_2024-09-22_1540.csv")
coaching_time = pd.read_excel("D:\\EIC-Code\\00-Python\\2024-08-Objective-identification-on-mental-health_CNN_LSTM\\data\\caoching_time.xlsx")

In [194]:
dailyactivity.rename(columns = {'ActivityDate':'date'}, inplace=True)
wear_time.rename(columns = {'Day':'date'}, inplace=True)
dailyFitbitActiveZoneMinutes.rename(columns = {'Date':'date'}, inplace=True)
dailyHRV.rename(columns = {'SleepDay':'date'}, inplace=True)
fitbitBreathingRate.rename(columns = {'SleepDay':'date'}, inplace = True)
fitbitSkinTemperature.rename(columns = {'SleepDay':'date'}, inplace=True)
sleepDay.rename(columns = {'SleepDay':'date'}, inplace=True)
sleepStageLogInfo.rename(columns = {'SleepDay':'date'}, inplace=True)
heartrate_15min.rename(columns = {'Time':'date'}, inplace=True)

#transfer date
total_data = [wear_time, dailyactivity, dailyFitbitActiveZoneMinutes, dailyHRV, fitbitBreathingRate, 
              fitbitSkinTemperature, sleepDay, sleepStageLogInfo, heartrate_15min]

for d in total_data:
    d['date'] = pd.to_datetime(d['date'], errors = 'coerce')
    d['date'] = d['date'].dt.date #.strftime('%m/%d/%Y').str.replace('/0', '/')

#transfer heart_rate_15 min
heartrate_15min = heartrate_15min.groupby(by = ['Id','date']).mean().reset_index()

C:\Users\karri\AppData\Local\Temp\ipykernel_29128\987032928.py:16: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  d['date'] = pd.to_datetime(d['date'], errors = 'coerce')
C:\Users\karri\AppData\Local\Temp\ipykernel_29128\987032928.py:16: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  d['date'] = pd.to_datetime(d['date'], errors = 'coerce')
C:\Users\karri\AppData\Local\Temp\ipykernel_29128\987032928.py:16: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  d['date'] = pd.to_datetime(d['date'], errors = 'coerce')


In [195]:
#check participants
#delete the date if total minutes wear time smaller than 10
# wear_time_larger_10_hours = wear_time[wear_time['TotalMinutesWearTime'] >= 10*60] #10hour, what's the meaning of four days a week, is this for calendar week or four consecutive days
# number_10 = wear_time_larger_10_hours['Id'].unique() #99, the number of total participants

# #delete the date if total minutes wear time smaller than 10
# wear_time_larger_0_hours = wear_time[wear_time['TotalMinutesWearTime'] >0] #10hour, what's the meaning of four days a week, is this for calendar week or four consecutive days
# number_0 = wear_time_larger_0_hours['Id'].unique() #99, the number of total participants

# total = wear_time['Id'].unique()
# for i in total:
#     if i not in number_10:
#         print(i)

In [196]:
merged1 = pd.merge(wear_time, dailyactivity, on=['Id','date'], how='left')  # inner join only keeps common dates
merged2 = pd.merge(merged1, sleepDay, on=['Id','date'], how='left')
merged3 = pd.merge(merged2, sleepStageLogInfo, on=['Id','date'], how='left')
merged4 = pd.merge(merged3, heartrate_15min, on=['Id','date'], how='left')
merged5 = pd.merge(merged4, dailyFitbitActiveZoneMinutes, on=['Id','date'], how='left')
merged6 = pd.merge(merged5, dailyHRV, on=['Id','date'], how='left')
merged7 = pd.merge(merged6, fitbitBreathingRate, on=['Id','date'], how='left')
merged_final = pd.merge(merged7, fitbitSkinTemperature, on=['Id','date'], how='left')

In [197]:

merged_final_10 = merged_final[(merged_final['TotalMinutesWearTime'] >= 10*60)] #10hour, what's the meaning of four days a week, is this for calendar week or four consecutive days
merged_0_3000steps = merged_final[(merged_final['TotalMinutesWearTime'] == 0) & (merged_final['TotalSteps'] >= 3000)]

In [198]:
merged_final = pd.concat([merged_final_10, merged_0_3000steps])
merged_final = merged_final.sort_values(by=['Id', 'date']) 

#for i in wear_time['Id'].unique():
#     if i not in merged_final['Id'].unique():
#         print(i) #test how many data deleted in this stage

In [199]:
#transfer Id to number only
merged_final['Id'] = [items[-3:] for items in merged_final['Id']]
merged_final['Id'] = [int(digit) for digit in merged_final['Id']]

Survey data, find sepecific coaching time

In [200]:
#check the number who complete consent form, baseline, health coaching, first follow-up, second follow-up

complete_var = ['consent_and_screening_survey_complete','baseline_wellness_behavior_survey_complete',
                 'health_coach_survey_complete', 'weeks_followup_survey_complete','weeks_followup_survey_96ac_complete'] #2 represent complete

for com in complete_var:
    print(com, len(survey_data[survey_data[com] == 2]))

consent_and_screening_survey_complete 175
baseline_wellness_behavior_survey_complete 129
health_coach_survey_complete 100
weeks_followup_survey_complete 96
weeks_followup_survey_96ac_complete 87


In [201]:
#survey data preprocessing
survey_data['health_coach_survey_complete'].unique()
survey_data_clean = survey_data[survey_data['health_coach_survey_complete'] == 2]#select the whose health coaching session is completed

time_stamp = ['consent_and_screening_survey_timestamp','baseline_wellness_behavior_survey_timestamp','postsession_survey_timestamp',
              'health_coach_survey_timestamp','weeks_followup_survey_timestamp','weeks_followup_survey_96ac_timestamp']

for time in time_stamp:
    survey_data_clean[time] = pd.to_datetime(survey_data_clean[time], errors = 'coerce' )
    survey_data_clean[time] = survey_data_clean[time].dt.date #.strftime('%m/%d/%Y').str.replace('/0', '/')

#transfer record ID
survey_data_clean.rename(columns = {'record_id':'Id'}, inplace=True)
survey_data_clean['Id'] = [int(digit) for digit in survey_data_clean['Id']]

C:\Users\karri\AppData\Local\Temp\ipykernel_29128\431148340.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  survey_data_clean[time] = pd.to_datetime(survey_data_clean[time], errors = 'coerce' )
C:\Users\karri\AppData\Local\Temp\ipykernel_29128\431148340.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  survey_data_clean[time] = survey_data_clean[time].dt.date #.strftime('%m/%d/%Y').str.replace('/0', '/')
C:\Users\karri\AppData\Local\Temp\ipykernel_29128\431148340.py:9: SettingWithCopyWarning: 
A valu

In [202]:
#coaching time data processing
coaching_time.rename(columns = {'Health Coach Session':'health_coaching_time', "ID":'Id'}, inplace=True)
coaching_time['health_coaching_time'] = pd.to_datetime(coaching_time['health_coaching_time'], errors = 'coerce' )
coaching_time['health_coaching_time'] = coaching_time['health_coaching_time'].dt.date

In [203]:
#merge it with survey data
survey_data_coaching = pd.merge(survey_data_clean, coaching_time, on=['Id'], how='left')

#math coaching time
time_stamp = ['Id','health_coaching_time', 'health_coach_survey_timestamp', 'postsession_survey_timestamp','weeks_followup_survey_timestamp', 'weeks_followup_survey_96ac_timestamp']
coaching_time_compare = survey_data_coaching[time_stamp]

Merge Fitbit data with coaching time

In [204]:
def assign_survey_date(row, time_lst):
    if pd.notna(time_lst) and time_lst == row['date']:
        return 'coach session'
    elif pd.notna(time_lst) and time_lst > row['date']:
        return 'baseline'
    elif pd.notna(time_lst) and time_lst < row['date']:
        return 'follow-up'
    else:
        return None  # In case no condition matches
    

def seven_day_average(data):

    length_baseline = len(data)
    #calculate the week value
    #average of total minutes wearatime
    data_mean = data.select_dtypes(include=[np.number]).mean()

    length_baseline = len(data)

    ndays = pd.Series({'ndays': length_baseline}) # add the number of days within this week
    data_mean = pd.concat([data_mean, ndays]) #the data wiht days

    return data_mean

def baseline(baseline_data):

    len_baseline = len(baseline_data)

    #calculate the baseline
    if len_baseline != 0:
        
        if len_baseline >= 7:
            all_index = list(baseline_data.index)
            index_last_seven_days = all_index[-7:]
            baseline_last_seven = baseline_data.loc[index_last_seven_days] #revise select data to basline
        else: 
            baseline_last_seven = baseline_data

        baseline_mean = seven_day_average(baseline_last_seven)
        baseline_mean.index = [str(col) + '_baseline' for col in baseline_mean.index] #revise the name
        
        return baseline_mean
    
    else:
        return None


def follow_Up(follow_up_data, coach_time):
    len_follow_up = len(follow_up_data)

    if len_follow_up!=0:
        last_day_date = follow_up_data[-1:]['date'] #last_day_date

        day_difference = (last_day_date.loc[last_day_date.index[0]] - coach_time.loc[coach_time.index[0]]).days
        
        time_loop = int(day_difference/7)+1

        next_7_days = coach_time #start day

        id_average = pd.Series()

        for i in range(1, time_loop+1):
            start_day = next_7_days
            
            start_index = start_day.index[0]

            next_7_days = start_day + pd.Timedelta(days=7)
            
            next_7day_data = follow_up_data[(follow_up_data['date'] > start_day[start_index]) & (follow_up_data['date']<= next_7_days[start_index])]
            
            follow_up_mean = seven_day_average(next_7day_data)

            follow_up_mean.drop(follow_up_mean.index[0], inplace = True)
            
            follow_up_mean.index = [str(col) + '_week' + str(i) for col in follow_up_mean.index] #revise the name
            
            id_average = pd.concat([id_average, follow_up_mean])
        
        return id_average

    else:
        return None


In [205]:
#add another column Fitbit data
fitbit_data = merged_final.copy()
fitbit_data.insert(loc = 2, column = 'survey_date', value = np.nan)

#Handle duplicate: replace 98 with  107 is the same person, 98 has fitbit data, 107 has survey data, we keep them consistent as 107
fitbit_data.loc[fitbit_data['Id'] == 98, 'Id'] = 107

#delete the participants who did not compplete health coaching session
survey_id = survey_data_clean ['Id'].unique() #participants who complete health coaching session
Fitbit_id = fitbit_data['Id'].unique() #participants who have fitbit data
no_completed_coaching = [] #[32, 80, 91, 163]

for id in Fitbit_id:
    if id not in survey_id:
        no_completed_coaching.append(id)

fitbit_data = fitbit_data[~fitbit_data['Id'].isin(no_completed_coaching)] 

In [206]:
#evaluate if there is basline or not
fitbit_id = fitbit_data['Id'].unique()
survey_id = survey_data_coaching['Id'].unique()

for i in range(len(fitbit_id)):

    individual_id = fitbit_id[i]
    if individual_id in survey_id:
        individual_survey_data = survey_data_coaching[survey_data_coaching['Id']==individual_id]
        ind = individual_survey_data.index[0]
        coach_time = individual_survey_data['health_coaching_time'][ind]

        individual_fitbit_data = fitbit_data[fitbit_data['Id']==individual_id]
        individual_fitbit_data['survey_date'] = individual_fitbit_data.apply(assign_survey_date, axis=1, time_lst = coach_time)
        
        fitbit_data.loc[fitbit_data['Id'] == individual_id, 'survey_date'] = individual_fitbit_data['survey_date'] #replace with new define

C:\Users\karri\AppData\Local\Temp\ipykernel_29128\3167257556.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  individual_fitbit_data['survey_date'] = individual_fitbit_data.apply(assign_survey_date, axis=1, time_lst = coach_time)


In [207]:
#delete participants who only have baseline data
#check the range of individual data
only_baseline = []

for i in fitbit_data['Id'].unique():
    if (fitbit_data[fitbit_data['Id']== i]['survey_date']=='baseline').all():
        length_id = len(fitbit_data[fitbit_data['Id']==i])
        only_baseline.append(i)
        #print(i, length_id)

fitbit_data = fitbit_data[~fitbit_data['Id'].isin(only_baseline)] #delete the data wiht only basline

In [208]:
#check the number of each individual
n_day = []
for i in fitbit_data['Id'].unique():
    n_day.append(len(fitbit_data[fitbit_data['Id']== i]))
    print(i, len(fitbit_data[fitbit_data['Id']== i]))

print(min(n_day), max(n_day))

5 60
7 59
8 32
13 26
14 72
15 35
17 136
18 90
20 86
21 116
22 97
23 51
24 163
25 82
26 154
27 68
28 140
31 65
35 118
38 127
39 101
41 143
43 33
45 126
48 24
50 81
51 117
52 50
54 62
56 133
57 125
59 123
60 66
61 118
62 134
65 50
67 87
68 17
69 28
70 7
73 92
74 60
76 111
77 59
81 79
84 59
86 40
87 70
88 110
89 66
94 92
95 109
97 79
107 138
100 88
101 66
102 84
106 64
108 34
109 64
113 58
117 53
118 57
119 59
120 61
122 91
123 65
124 65
125 91
127 41
129 91
130 62
131 55
132 79
133 62
135 52
136 75
137 76
138 59
141 73
142 49
143 29
144 67
146 83
147 62
151 28
153 64
156 34
162 76
164 27
167 48
7 163


In [209]:
#used variables
used_var = ['Id','date','survey_date','TotalMinutesWearTime', 'TotalSteps', 'VeryActiveMinutes', 'FairlyActiveMinutes',
       'LightlyActiveMinutes', 'SedentaryMinutes', 'RestingHeartRate','TotalActiveZoneMinutes', 'FatBurnActiveZoneMinutes',
       'CardioActiveZoneMinutes', 'PeakActiveZoneMinutes', 'DailyRMSSD', 'DeepRMSSD','AvgBreathsPerMinute',
       'TotalMinutesAsleep_x', 'TotalTimeInBed_x', 'TotalTimeAwake', 'TotalMinutesLight', 
       'TotalMinutesDeep','TotalMinutesREM'] #delete 'Above', 'Below', Custom Zone

used_fitbit_data = fitbit_data[used_var]

used_fitbit_data.rename(columns = {'TotalMinutesAsleep_x':'TotalMinutesAsleep',
                         'TotalTimeInBed_x':'TotalMinutesinbed',
                         'TotalTimeAwake': 'TotalMinutesAwake'}, inplace = True)

all_ids = used_fitbit_data['Id'].unique()

fitbit_final_output = pd.DataFrame()

#main loop
for id in all_ids:
    select_data = used_fitbit_data[used_fitbit_data['Id'] == id]
    sort_select_data = select_data.sort_values(by='date') #sort select data

    coach_time = coaching_time_compare[coaching_time_compare['Id'] == id]['health_coaching_time']

    baseline_data = sort_select_data[sort_select_data['survey_date'] == 'baseline']
    follow_up_data = sort_select_data[sort_select_data['survey_date'] == 'follow-up']

    baseline_mean = baseline(baseline_data)
    id_average = follow_Up(follow_up_data, coach_time)

    baseline_follow_up = pd.concat([baseline_mean,id_average])
    trans_baseline_follow_up = pd.DataFrame(baseline_follow_up).T
    
    #trans_baseline_follow_up['id'] = id
    fitbit_final_output = pd.concat([fitbit_final_output, trans_baseline_follow_up])

C:\Users\karri\AppData\Local\Temp\ipykernel_29128\162089700.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  used_fitbit_data.rename(columns = {'TotalMinutesAsleep_x':'TotalMinutesAsleep',


In [210]:
fitbit_final_output.to_csv('fitbit_final_output_v2.csv')

Merge fitbit data with survey data

In [211]:
#drop non-use column name
survey_new_data = survey_data_coaching.drop(columns = ['consent_and_screening_survey_timestamp', 
                                                     'informed_consent', 
                                                     'redcap_survey_identifier', 
                                                        'health_coach_survey_timestamp', 
                                                        'postsession_survey_timestamp',
                                                        'consent_and_screening_survey_complete',
                                                        'initials',
                                                        'Participants Name',
                                                        'initials_follow_up',
                                                        'email_address',
                                                        'name',
                                                        'health_coach_survey_complete',
                                                        'initials_post',
                                                        'postsession_survey_complete',
                                                        'demographics_zip',
                                                        'demographics_ziphome',
                                                        'recruitment',
                                                        'student_id',
                                                        'initials_follow_up_v2'

                                                        ])

In [212]:
fitbit_final_output.rename(columns = {'Id_baseline':'Id'}, inplace=True)
survey_fitbit = pd.merge(survey_new_data, fitbit_final_output, on = ['Id'], how = 'left')

#check how many participants in Redcap, but not in fitbit
# for i in survey_new_data['Id'].unique():
#     if i not in fitbit_final_output['Id'].unique():
#         print(i)

In [213]:
survey_fitbit.to_csv('final_survey_fitbit_v2.csv')